In [1]:
import pandas as pd
OUTPUT_DATABASE = "../data/profile_layers.csv"

# Transformers models characteristics

In [2]:
df_trans = pd.read_csv(OUTPUT_DATABASE)

In [3]:
df_trans

,net,layer,layer_params,depth,output_size
0,resnet50d,Conv2d,864,2,401408
1,resnet50d,BatchNorm2d,64,2,401408
2,resnet50d,ReLU,0,2,401408
3,resnet50d,Conv2d,9216,2,401408
4,resnet50d,BatchNorm2d,64,2,401408
...,...,...,...,...,...
8563,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,Dropout,0,4,787200
8564,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,Identity,0,3,787200
8565,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,Identity,0,1,787200
8566,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,LayerNorm,1536,1,768


In [4]:
df_trans["para_by_depth"] = df_trans["layer_params"] * df_trans["depth"]

In [ ]:
details_df = df_trans.groupby(["net", "layer"]).describe()


In [ ]:
details_df["counter"] = details_df.loc[:, pd.IndexSlice["layer_params", "count"]]


In [ ]:
details_df = details_df.drop(["count", "std", "25%", "50%", "75%"], level=1, axis="columns")

In [ ]:
details_df.to_excel("/tmp/profile.xlsx")